<a href="https://colab.research.google.com/github/newcolour1994/3d_DenseNet_mri/blob/master/DialoGPT_gen_topk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch
!pip install transformers

     |████████████████████████████████| 317kB 5.0MB/s 
     |████████████████████████████████| 1.0MB 43.5MB/s 
     |████████████████████████████████| 860kB 29.4MB/s 
     |████████████████████████████████| 645kB 41.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=2e9545122d2c350bf5fb3312ea5ed0a6adb99a7bf06b3c757fac0688f7deaeaa
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses


In [ ]:
!wget https://convaisharables.blob.core.windows.net/lsp/multiref/medium_ft.pkl

--2019-11-08 13:58:50--  https://convaisharables.blob.core.windows.net/lsp/multiref/medium_ft.pkl
Resolving convaisharables.blob.core.windows.net (convaisharables.blob.core.windows.net)... 13.77.184.64
Connecting to convaisharables.blob.core.windows.net (convaisharables.blob.core.windows.net)|13.77.184.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862954531 (823M) [application/octet-stream]
Saving to: ‘medium_ft.pkl’

medium_ft.pkl       100%[===================>] 822.98M  30.2MB/s    in 18s     

2019-11-08 13:59:08 (46.4 MB/s) - ‘medium_ft.pkl’ saved [862954531/862954531]



In [ ]:
import os
import torch
import torch.nn.functional as F
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
weights = torch.load('medium_ft.pkl')
medium_config = GPT2Config(n_embd=1024,n_layer=24,n_head=16)
model = GPT2LMHeadModel(medium_config)

# fix misused key value
weights["lm_head.weight"] = weights["lm_head.decoder.weight"]
weights.pop("lm_head.decoder.weight",None)

model.load_state_dict(weights)
model.eval()
model.to('cuda')

conditioned_tokens = []
generated_tokens = []


def reinput(text):
	global conditioned_tokens
	os.system('cls' if os.name == 'nt' else 'clear')
	conditioned_tokens = tokenizer.encode(text) + [50256]
	print("\nUser input:\n" + text + "\n")


def top_p_filtering(logits, top_p=0.9, filter_value=-float('Inf')):
  """
  Credit: https://gist.github.com/thomwolf/1a5a29f6962089e871b94cbd09daf317
  """
  assert logits.dim() == 1  # batch size 1 for single word generation
  sorted_logits, sorted_indices = torch.sort(logits, descending=True)
  cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)
  # remove tokens with cumulative probability above the threshold
  sorted_indices_to_remove = cumulative_probs > top_p
  # shift the indices to the right to keep also the first token above the threshold
  sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
  sorted_indices_to_remove[..., 0] = 0
  indices_to_remove = sorted_indices[sorted_indices_to_remove]
  logits[indices_to_remove] = filter_value
  return logits


def recalc():
	global conditioned_tokens
	global generated_tokens
  # for segment display purpose, keep 2 sets of tokens
	indexed_tokens = conditioned_tokens + generated_tokens
	tokens_tensor = torch.tensor([indexed_tokens])
	tokens_tensor = tokens_tensor.to('cuda')
	with torch.no_grad():
	    outputs = model(tokens_tensor)
	    predictions = outputs[0]
	logits = predictions[0, -1, :]
	filtered_logits = top_p_filtering(logits)
	probabilities = F.softmax(filtered_logits, dim=-1)
	next_token = torch.multinomial(probabilities, 1)
	generated_tokens.append(next_token.item())
	return next_token.item()

def generate():
	global conditioned_tokens
	global generated_tokens
	while True:
		result = recalc()
		if result == 50256:
      # end-of-text : 50256
      # use this special token to split segments
			print(tokenizer.decode(generated_tokens[:-1]))
			conditioned_tokens += generated_tokens
			generated_tokens = []
			break

100%|██████████| 456318/456318 [00:00<00:00, 1462961.10B/s]


How to use:

This script will automatically generate a complete segment(or a post if you wish) and wait for user input.
1. Press Enter (input nothing) to keep generating under current topic.
2. Type in prompt text to start another topic.



In [ ]:
reinput("Do you know James Blunt?")
generate()

while True:
	cmd = input()
	if cmd != "":	reinput(cmd)
	generate()


User input:
Do you know James Blunt?

You know him too?

James Blunt was probably one of the few guys I worked with at the time.

You worked with him? I think you should send him a PM

Only if you post the contact info here too.


KeyboardInterrupt: ignored